## Import

In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models.segmentation as segmentation
import segmentation_models_pytorch as smp
import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision.transforms as transforms
from albumentations.core.composition import Compose

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Fuctions

## RLE decoding/encoding

In [2]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

## Model load

In [3]:
# save_dir = "./path/save/resnet50_l2/"  # 모델 저장 디렉토리
# model_name = "deeplabv3_resnet50_l2_trained_epoch{}.pth"  # 모델 파일 이름 패턴

# # 훈련된 모델을 저장하는 함수
# def save_model(model, epoch):
#     save_path = save_dir + model_name.format(epoch)
#     torch.save(model.state_dict(), save_path)
#     print(f"Epoch {epoch} 모델 저장이 완료되었습니다.")

# 모델 불러오는 함수
def load_model(model, load_path):
    state_dict = torch.load(load_path)
    # 이전에 저장된 모델과 현재 모델 간 레이어 일치 여부 확인
    model_dict = model.state_dict()
    new_state_dict = {k: v for k, v in state_dict.items() if k in model_dict}
    model_dict.update(new_state_dict)
    model.load_state_dict(model_dict)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print("모델 불러오기가 완료되었습니다.")

In [6]:
class SatelliteDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False): # transform 전처리하거나 다른 형태로 변환하는데 사용, infer 데이터를 추론 모드로 설정할지 여부
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1] # 해당 인덱스에 위치한 데이터프레임의 두 번째 열(column)에 있는 이미지 경로를 가져옴
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_rle = self.data.iloc[idx, 2] # 데이터프레임의 idx 행에서 세 번째 열(column)에 있는 마스크 정보를 가져옴
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1])) # 원래의 마스크 이미지로 변환

        if self.transform:
            augmented = self.transform(image=image, mask=mask) # 이미지와 마스크를 변환
            image = augmented['image'] # 변환된 이미지를 딕셔너리에서 가져와 image 변수에 할당
            mask = augmented['mask'] # 변환된 이미지를 딕셔너리에서 가져와 mask 변수에 할당

        return image, mask

# Train Data Loader

In [4]:
# albumentations 라이브러리를 사용하여 이미지 데이터에 대한 변환(transform) 파이프라인 정의
transform = A.Compose(
    [
        # A.Resize(224, 224), # 이미지 크기 조정
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 이미지 픽셀값 정규화
        ToTensorV2() # 이미지를 텐서로 변환
    ]
)

# # 데이터 증강을 위한 transform 파이프라인 정의
# transform = A.Compose(
#     [
#         A.Resize(224, 224),  # 이미지 크기 조정
#         A.RandomCrop(224, 224),  # 랜덤한 위치에서 224x224 크기로 자르기
#         A.RandomRotate90(),  # 90도 회전 (랜덤하게)
#         A.HorizontalFlip(p=0.5),  # 수평 뒤집기 확률 50%
#         A.VerticalFlip(p=0.5),  # 수직 뒤집기 확률 50%
#         A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.7),  # 이동, 크기 조정, 회전
#         A.CLAHE(p=0.2),  # CLAHE를 통한 대비 개선
#         A.Cutout(num_holes=8, max_h_size=8, max_w_size=8, p=0.5),  # Cutout 적용
#         A.Normalize(),  # 이미지 픽셀값 정규화
#         ToTensorV2()  # 이미지를 텐서로 변환
#     ]
# )

# dataset = SatelliteDataset(csv_file='./train.csv', transform=transform) # dataset 불러오기
# dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Test Data Loader

In [113]:
test_dataset = SatelliteDataset(csv_file='./test.csv', transform=transform, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)
print(len(test_dataset))
print(len(test_dataloader))

60640
3790


# Load Model

In [114]:
# # 모델 생성
# model1 = smp.Unet(encoder_name="resnet50",  # 필수 파라미터: 사용할 인코더 백본의 이름
#     in_channels=3,    # 필수 파라미터: 입력 이미지의 채널 수 (일반적으로 3(RGB) 또는 1(Grayscale))
#     classes=1,        # 필수 파라미터: 세그멘테이션 클래스의 수 (예: 물체 탐지의 경우 물체 클래스 수)
#     encoder_weights="imagenet"  # 선택적 파라미터: 사용할 사전 훈련된 인코더 가중치의 경로 또는 'imagenet'으로 설정하여 ImageNet 가중치 사용
# )

# DeepLabV3+ 모델 정의
model1 = smp.PSPNet(encoder_name="resnet152",  # 필수 파라미터: 사용할 인코더 백본의 이름
    in_channels=3,    # 필수 파라미터: 입력 이미지의 채널 수 (일반적으로 3(RGB) 또는 1(Grayscale))
    classes=1,        # 필수 파라미터: 세그멘테이션 클래스의 수 (예: 물체 탐지의 경우 물체 클래스 수)
    encoder_weights="imagenet"  # 선택적 파라미터: 사용할 사전 훈련된 인코더 가중치의 경로 또는 'imagenet'으로 설정하여 ImageNet 가중치 사용
)

model2 = smp.PSPNet(
    encoder_name="densenet161",   # 백본으로 ResNet-50 사용
    encoder_weights="imagenet", # ImageNet 가중치로 초기화
    in_channels=3,             # 입력 이미지 채널 수 (RGB 이미지인 경우 3)
    classes=1                  # 출력 클래스 수 (이진 분류인 경우 1)
)

model3 = smp.Unet(encoder_name="resnet50",  # 필수 파라미터: 사용할 인코더 백본의 이름
    in_channels=3,    # 필수 파라미터: 입력 이미지의 채널 수 (일반적으로 3(RGB) 또는 1(Grayscale))
    classes=1,        # 필수 파라미터: 세그멘테이션 클래스의 수 (예: 물체 탐지의 경우 물체 클래스 수)
    encoder_weights="imagenet"  # 선택적 파라미터: 사용할 사전 훈련된 인코더 가중치의 경로 또는 'imagenet'으로 설정하여 ImageNet 가중치 사용
)

model4 = smp.DeepLabV3Plus(encoder_name="timm-mobilenetv3_large_100",  # 필수 파라미터: 사용할 인코더 백본의 이름
    in_channels=3,    # 필수 파라미터: 입력 이미지의 채널 수 (일반적으로 3(RGB) 또는 1(Grayscale))
    classes=1,        # 필수 파라미터: 세그멘테이션 클래스의 수 (예: 물체 탐지의 경우 물체 클래스 수)
    encoder_weights="imagenet"  # 선택적 파라미터: 사용할 사전 훈련된 인코더 가중치의 경로 또는 'imagenet'으로 설정하여 ImageNet 가중치 사용
)

model5 = smp.DeepLabV3Plus(encoder_name="timm-mobilenetv3_large_100",  # 필수 파라미터: 사용할 인코더 백본의 이름
    in_channels=3,    # 필수 파라미터: 입력 이미지의 채널 수 (일반적으로 3(RGB) 또는 1(Grayscale))
    classes=1,        # 필수 파라미터: 세그멘테이션 클래스의 수 (예: 물체 탐지의 경우 물체 클래스 수)
    encoder_weights="imagenet"  # 선택적 파라미터: 사용할 사전 훈련된 인코더 가중치의 경로 또는 'imagenet'으로 설정하여 ImageNet 가중치 사용
)

model6 = segmentation.deeplabv3_resnet50(pretrained=True) # 사전 훈련된 deeplabv3 모델 가져옴, pretrained=True이면 모델이 사전 훈련된 가중치 사용하여 초기화됨
model6.classifier[4] = nn.Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1)) # 1*1 컨볼루션 레이어 생성, 입력 채널 256, 출력 채널 1로 설정
# model = model.to(device)

# 저장된 모델의 파라미터 불러오기 (strict=False 옵션 사용)
state_dict_1 = torch.load('./path/save/ensemble/psp_resnet_trained_epoch13.pth', map_location=torch.device('cpu'))
state_dict_2 = torch.load('./path/save/ensemble/psp_dense_base_trained_epoch55.pth', map_location=torch.device('cpu'))
state_dict_3 = torch.load('./path/save/ensemble/unet_resnet_50_new_aug_noempty_trained_epoch32.pth', map_location=torch.device('cpu'))
state_dict_4 = torch.load('./path/save/ensemble/v3plus_mobilenet_epoch42.pth', map_location=torch.device('cpu'))
state_dict_5 = torch.load('./path/save/ensemble/mobileNet_focal_dice_trained_epoch23.pth', map_location=torch.device('cpu'))
state_dict_6 = torch.load('./path/save/ensemble/deeplabv3_resnet50_focal_dice_batch8_trained_epoch40.pth', map_location=torch.device('cpu'))

# # 저장된 모델의 클래스 수 (1개의 클래스일 때)
# saved_num_classes = 1

# # 현재 모델의 클래스 수 (예시로 21로 설정, 실제 사용하는 클래스 수로 수정)
# current_num_classes = 1

# # 모델의 분류기 레이어 크기 변경
# if saved_num_classes != current_num_classes:
#     # 모델의 분류기 레이어를 1x1 컨볼루션 레이어로 수정
#     model.classifier[4] = torch.nn.Conv2d(256, current_num_classes, kernel_size=(1, 1), stride=(1, 1))
#     # 모델의 분류기 레이어를 초기화
#     torch.nn.init.xavier_uniform_(model.classifier[4].weight)  # 또는 다른 초기화 방법 사용

# 모델의 파라미터 로드
model1.load_state_dict(state_dict_1, strict=False)
model2.load_state_dict(state_dict_2, strict=False)
model3.load_state_dict(state_dict_3, strict=False)
model4.load_state_dict(state_dict_4, strict=False)
model5.load_state_dict(state_dict_5, strict=False)
model6.load_state_dict(state_dict_6, strict=False)

# GPU 사용이 가능한 경우에는 GPU로 데이터 이동
device1 = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device2 = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device3 = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device4 = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device5 = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device6 = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# b5=b5Model.to(b5Device)
# mob=mobModel.to(mobDevice)
# print(b5+mob)
model1.to(device1)
model2.to(device2)
model3.to(device3)
model4.to(device4)
model5.to(device4)
model6.to(device4)
# print(mobModel.to(mobDevice))
# print(b5Model.to(b5Device))

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

# Inference

In [110]:
# 결과를 저장할 리스트 초기화
result1 = []
result2=[]
result3=[]
result4=[]
result5=[]
result6=[]

all_result=[]
or_result=[]
and_result=[]
vote_result=[]

with torch.no_grad(): # 역전파 비활성화, 파라미터 업데이트 금지
    # print(len(test_dataloader))
    for images in tqdm(test_dataloader): # 데이터 로드
        images = images.float().to(device) 

        outputs1 = model1(images) # 테스트 이미지 전달하여 예측 결과 얻음
        outputs2 = model2(images) # 테스트 이미지 전달하여 예측 결과 얻음
        outputs3 = model3(images) # 테스트 이미지 전달하여 예측 결과 얻음
        outputs4 = model4(images) # 테스트 이미지 전달하여 예측 결과 얻음
        outputs5 = model5(images) # 테스트 이미지 전달하여 예측 결과 얻음
        outputs6 = model6(images)['out'] # 테스트 이미지 전달하여 예측 결과 얻음

        masks1 = torch.sigmoid(outputs1).cpu().numpy() # outputs는 모델 예측 결과, 확률값으로 변환하기 위해 시그모이드 함수 적용한 후 각 픽셀 값을 0과 1사이의 확률값으로 변환하고, 넘파이 배열로 변환
        masks2 = torch.sigmoid(outputs2).cpu().numpy() # outputs는 모델 예측 결과, 확률값으로 변환하기 위해 시그모이드 함수 적용한 후 각 픽셀 값을 0과 1사이의 확률값으로 변환하고, 넘파이 배열로 변환
        masks3 = torch.sigmoid(outputs3).cpu().numpy() # outputs는 모델 예측 결과, 확률값으로 변환하기 위해 시그모이드 함수 적용한 후 각 픽셀 값을 0과 1사이의 확률값으로 변환하고, 넘파이 배열로 변환
        masks4 = torch.sigmoid(outputs4).cpu().numpy() # outputs는 모델 예측 결과, 확률값으로 변환하기 위해 시그모이드 함수 적용한 후 각 픽셀 값을 0과 1사이의 확률값으로 변환하고, 넘파이 배열로 변환
        masks5 = torch.sigmoid(outputs5).cpu().numpy() # outputs는 모델 예측 결과, 확률값으로 변환하기 위해 시그모이드 함수 적용한 후 각 픽셀 값을 0과 1사이의 확률값으로 변환하고, 넘파이 배열로 변환
        masks6 = torch.sigmoid(outputs6).cpu().numpy() # outputs는 모델 예측 결과, 확률값으로 변환하기 위해 시그모이드 함수 적용한 후 각 픽셀 값을 0과 1사이의 확률값으로 변환하고, 넘파이 배열로 변환

        # 두 모델의 예측 결과를 다수결 방식으로 합치기
        ensemble_predictions = (outputs1 + outputs2 + outputs3 + outputs4 + outputs5+outputs6) >= 5  # 두 모델의 예측이 1 이상이면 True, 아니면 False

        allMasks= masks1*0.16+masks2*0.37+masks3*0.16+masks4*0.16+masks5*0.15
        # allMasks= masks2*0.65+masks6*0.35
        # allMasks= ((masks1*0.7+masks2*0.2)+(masks1*0.8+masks3*0.2))/2

        masks1 = np.squeeze(masks1, axis=1) # 불필요한 차원 제거
        masks1 = (masks1 > 0.35).astype(np.uint8) # 최종 이진화 예측 마스크 얻음
        
        masks2 = np.squeeze(masks2, axis=1) # 불필요한 차원 제거
        masks2 = (masks2 > 0.35).astype(np.uint8) # 최종 이진화 예측 마스크 얻음

        masks3 = np.squeeze(masks3, axis=1) # 불필요한 차원 제거
        masks3 = (masks3 > 0.35).astype(np.uint8) # 최종 이진화 예측 마스크 얻음

        masks4 = np.squeeze(masks4, axis=1) # 불필요한 차원 제거
        masks4 = (masks4 > 0.35).astype(np.uint8) # 최종 이진화 예측 마스크 얻음

        masks5 = np.squeeze(masks5, axis=1) # 불필요한 차원 제거
        masks5 = (masks5 > 0.35).astype(np.uint8) # 최종 이진화 예측 마스크 얻음

        masks6 = np.squeeze(masks6, axis=1) # 불필요한 차원 제거
        masks6 = (masks6 > 0.35).astype(np.uint8) # 최종 이진화 예측 마스크 얻음
        
        # print(len(images))
        allMasks = np.squeeze(allMasks, axis=1) # 불필요한 차원 제거
        allMasks = (allMasks > 0.30).astype(np.uint8) # 최종 이진화 예측 마스크 얻음

        # # print(type(b5Masks))
        # orMasks=np.logical_or(masks1,masks2)
        # orMasks=np.logical_or(orMasks,masks3)
        # orMasks=np.logical_or(orMasks,masks4)

        # andMasks=np.logical_and(masks1,masks2)
        # andMasks=np.logical_and(andMasks,masks3)
        # andMasks=np.logical_and(andMasks,masks3)

        # for i in range(len(images)):
        #     mask_rle1 = rle_encode(masks1[i]) # RLE로 변환, mask_rle에 인코딩 결과 저장

        #     if mask_rle1 == '':
        #         result1.append(-1) # 빌딩 없으면 -1 저장
        #     else:
        #         result1.append(mask_rle1) # 아니면 mask_rle 저장

        # for i in range(len(images)):
        #     mask_rle2 = rle_encode(masks2[i]) # RLE로 변환, mask_rle에 인코딩 결과 저장

        #     if mask_rle2 == '':
        #         result2.append(-1) # 빌딩 없으면 -1 저장
        #     else:
        #         result2.append(mask_rle2) # 아니면 mask_rle 저장

        for i in range(len(images)):
            all_mask_rle = rle_encode(allMasks[i]) # RLE로 변환, mask_rle에 인코딩 결과 저장

            if all_mask_rle == '':
                all_result.append(-1) # 빌딩 없으면 -1 저장
            else:
                all_result.append(all_mask_rle) # 아니면 mask_rle 저장
        

        # for i in range(len(images)):
        #     or_mask_rle = rle_encode(orMasks[i]) # RLE로 변환, mask_rle에 인코딩 결과 저장

        #     if or_mask_rle == '':
        #         or_result.append(-1) # 빌딩 없으면 -1 저장
        #     else:
        #         or_result.append(or_mask_rle) # 아니면 mask_rle 저장

        # for i in range(len(images)):
        #     and_mask_rle = rle_encode(andMasks[i]) # RLE로 변환, mask_rle에 인코딩 결과 저장

        #     if and_mask_rle == '':
        #         and_result.append(-1) # 빌딩 없으면 -1 저장
        #     else:
        #         and_result.append(and_mask_rle) # 아니면 mask_rle 저장     

        for i in range(len(images)):
            ensemble_predictions_cpu = ensemble_predictions[i].cpu()  # Move the tensor to CPU
            vote_mask_rle = rle_encode(ensemble_predictions_cpu)  # RLE로 변환, mask_rle에 인코딩 결과 저장

            if vote_mask_rle == '':
                vote_result.append(-1)  # 빌딩 없으면 -1 저장
            else:
                vote_result.append(vote_mask_rle)  # 아니면 mask_rle 저장

            visualized_image = images[i].cpu().numpy().transpose((1, 2, 0)) # 이미지 시각화하기 위해 넘파이 배열로 가져옴
            masks_visualized1 = masks1[i] * 255 # 이진화 마스크로 변환
            masks_visualized2 = masks2[i] * 255 # 이진화 마스크로 변환
            masks_visualized3 = masks3[i] * 255 # 이진화 마스크로 변환
            masks_visualized4 = masks4[i] * 255 # 이진화 마스크로 변환
            masks_visualized5 = masks5[i] * 255 # 이진화 마스크로 변환

            pre_masks_visualized = ensemble_predictions_cpu.numpy().squeeze() * 255  # 이진화 마스크로 변환
            all_masks_visualized = allMasks[i] * 255 # 이진화 마스크로 변환
            # or_masks_visualized = orMasks[i] * 255 # 이진화 마스크로 변환
            # and_masks_visualized = andMasks[i] * 255 # 이진화 마스크로 변환

            # plt.figure(figsize=(15, 10))  # 원하는 크기로 조정

            # plt.subplot(1, 7, 1)
            # plt.imshow(visualized_image)
            # plt.title("Input Image")

            # plt.subplot(1, 7, 2)
            # plt.imshow(masks_visualized1, cmap='gray')
            # plt.title("152")

            # plt.subplot(1, 7, 3)
            # plt.imshow(masks_visualized2, cmap='gray')
            # plt.title("dense")

            # plt.subplot(1, 7, 4)
            # plt.imshow(masks_visualized3, cmap='gray')
            # plt.title("50")

            # plt.subplot(1, 7, 5)
            # plt.imshow(masks_visualized4, cmap='gray')
            # plt.title("mobilenet")

            # plt.subplot(1, 7, 6)
            # plt.imshow(pre_masks_visualized, cmap='gray')
            # plt.title("vote")

            # plt.subplot(1, 7, 7)
            # plt.imshow(all_masks_visualized, cmap='gray')
            # plt.title("all")
            # plt.subplot(1, 6, 5)
            # plt.imshow(or_masks_visualized, cmap='gray')
            # plt.title("or Mask")

            # plt.subplot(1, 6, 6)
            # plt.imshow(and_masks_visualized, cmap='gray')
            # plt.title("and Mask")

            # plt.show()

100%|██████████| 7/7 [00:02<00:00,  3.14it/s]


# Submission

In [111]:
submit = pd.read_csv('./validation_sample.csv')
submit['mask_rle'] = all_result

In [112]:
submit.to_csv('./152_dense_50_v3plus_v3/all_validation_submit.csv', index=False)

In [115]:
submit = pd.read_csv('./validation_sample.csv')
submit['mask_rle'] = vote_result

In [116]:
submit.to_csv('./152_dense_50_v3plus_v3/vote_validation_submit.csv', index=False)

In [17]:
submit = pd.read_csv('./validation_sample.csv')
submit['mask_rle'] = or_result

In [18]:
submit.to_csv('./psp_resnet_dense_unet_resnet_vote/or_validation_submit.csv', index=False)

In [19]:
submit = pd.read_csv('./validation_sample.csv')
submit['mask_rle'] = and_result


In [20]:
submit.to_csv('./psp_resnet_dense_unet_resnet_vote/and_validation_submit.csv', index=False)